## Airport Filtering

In [16]:
import connectionMongoDB as mongo
import json
import pandas as pd
import getLocation as gl
from haversine import haversine,Unit
import geoqueryfunction as gq

- All information from airports.csv from: https://ourairports.com/data/

In [17]:
db_airp, coll_airp = mongo.connectCollection('companies','airport')

In [18]:
airport_locations = list(coll_airp.find())


- Adding geolocations to the large airports:

In [19]:
for a in airport_locations:
    long=a['longitude_deg']
    lat=a['latitude_deg']
    value = {"$set": {'location':gl.getLocation(long,lat)}}
    coll_airp.update_one(a,value)

In [20]:
len(airport_locations)


615

- Exporting json with schools, starbucks and companies with more than 1M$ raised.

In [21]:
with open('./output/StarbucksSchool_1M.json', 'r') as myfile:
    data=myfile.read()
obj = json.loads(data)
merge3=pd.DataFrame(obj)

In [22]:
merge3_df=pd.DataFrame(merge3)

In [23]:
airports_df=pd.DataFrame(airport_locations)

In [24]:
merge3_df['airports']=merge3_df.apply(lambda a: gq.geoquery(coll_airp,a['Longitude'],a['Latitude'],20000),axis=1)


In [25]:
merge3_df=merge3_df[(merge3_df['airports'].notnull())]

In [28]:
merge3_df.shape

(727, 21)